## 1.1. Linear Models

### 1.1.3. Lasso

The [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso) is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer non-zero coefficients, effectively reducing the number of features upon which the given solution is dependent. For this reason, Lasso and its variants are fundamental to the field of compressed sensing. Under certain conditions, it can recover the exact set of non-zero coefficients (see [Compressive sensing: tomography reconstruction with L1 prior (Lasso)](https://scikit-learn.org/stable/auto_examples/applications/plot_tomography_l1_reconstruction.html#sphx-glr-auto-examples-applications-plot-tomography-l1-reconstruction-py)).

Mathematically, it consists of a linear model with an added regularization term. The objective function to minimize is:

$$\min_w\frac{1}{2n_{samples}}\lVert Xw - y \lVert_2^2 + \alpha\lVert w \rVert_1$$

The lasso estimate thus solves the minimization of the least-squares penalty with $\alpha\lVert w \rVert_1$ added, where $\alpha$ is a constant and $\lVert w \rVert_1$ is the $\ell_1$-norm of the coefficient vector.

The implementation in the :class:`~sklearn.linear_model.Lasso` uses coordinate descent as the algorithm to fit the coefficients. See [Least Angle Regression](https://scikit-learn.org/stable/modules/linear_model.html#least-angle-regression) for another implementation:

In [1]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit([[0, 0], [1, 1]], [0, 1])
reg.predict([[1, 1]])

array([0.8])

The function :class:`~sklearn.linear_model.lasso_path` is useful for lower-level tasks, as it computes the coefficients along the full path of possible values.

**Example:**
- L1-based models for Sparse Signals - [Sci-kit Link](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_and_elasticnet.html#sphx-glr-auto-examples-linear-model-plot-lasso-and-elasticnet-py) | [Python code](https://github.com/baksho/ml-handson/blob/main/Examples/06_example_plot_lasso_and_elasticnet.py) | [Jupyter Notebook](https://github.com/baksho/ml-handson/blob/main/Examples/06_example_plot_lasso_and_elasticnet.ipynb)
- Compressive sensing: tomography reconstruction with L1 prior (Lasso) - [Sci-kit Link](https://scikit-learn.org/stable/auto_examples/applications/plot_tomography_l1_reconstruction.html#sphx-glr-auto-examples-applications-plot-tomography-l1-reconstruction-py) | [Python code](https://github.com/baksho/ml-handson/blob/main/Examples/07_example_plot_tomography_l1_reconstruction.py) | [Jupyter Notebook](https://github.com/baksho/ml-handson/blob/main/Examples/07_example_plot_tomography_l1_reconstruction.ipynb)
- Common pitfalls in the interpretation of coefficients of linear models - [Sci-kit Link](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#sphx-glr-auto-examples-inspection-plot-linear-model-coefficient-interpretation-py) | [Python code](https://github.com/baksho/ml-handson/blob/main/Examples/05_example_plot_linear_model_coefficient_interpretation.py) | [Jupyter Notebook](https://github.com/baksho/ml-handson/blob/main/Examples/05_example_plot_linear_model_coefficient_interpretation.ipynb)